<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AUTOGEN_GOOGLEADK_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env -q
!pip install -U autogen-agentchat autogen-ext[openai] -q

import colab_env
#import os
#key_api = os.environ.get("OPENAI_API_KEY")

## Autogen

In [31]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


def calculator(a: float, b: float, operator: str) -> str:
    """Perform basic arithmetic operations."""
    try:
        if operator == '+':
            return str(a + b)
        elif operator == '-':
            return str(a - b)
        elif operator == '*':
            return str(a * b)
        elif operator == '/':
            if b == 0:
                return 'Error: Division by zero'
            return str(a / b)
        else:
            return 'Error: Invalid operator. Please use +, -, *, or /'
    except Exception as e:
        return f'Error: {str(e)}'


async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o-2024-11-20")
    termination = MaxMessageTermination(
        max_messages=10) | TextMentionTermination("TERMINATE")
    assistant = AssistantAgent(
        "assistant", model_client=model_client, tools=[calculator])
    team = RoundRobinGroupChat([assistant], termination_condition=termination)
    await Console(team.run_stream(task="What is the result of 545.34567 * 34555.34"))

await main()

---------- user ----------
What is the result of 545.34567 * 34555.34
---------- assistant ----------
[FunctionCall(id='call_bSUPFy57VHxeLhtFzq3b7Ibu', arguments='{"a":545.34567,"b":34555.34,"operator":"*"}', name='calculator')]
---------- assistant ----------
[FunctionExecutionResult(content='18844605.0443778', name='calculator', call_id='call_bSUPFy57VHxeLhtFzq3b7Ibu', is_error=False)]
---------- assistant ----------
18844605.0443778
---------- assistant ----------
The result of \( 545.34567 \times 34555.34 \) is approximately \( 18,844,605.0444 \). TERMINATE


## Goggle ADK

In [27]:
!pip install google_adk -q

In [32]:
from typing import Any, Callable, List
import sys
import asyncio


class Agent:
    def __init__(self, name, model, description, instruction, tools: List[Callable[..., Any]]):
        self.name = name
        self.model = model
        self.description = description
        self.instruction = instruction
        self.tools = tools

    async def run(self, task: str) -> str:
        """Executes the agent on the given task.

        Args:
            task: The task for the agent to perform.

        Returns:
            The agent's response.
        """
        print(f"Agent {self.name} received task: {task}")
        sys.stdout.flush()

        # Hypothetical implementation to extract calculation operations
        if any(op in task for op in ['+', '-', '*', '/']):
            try:
                parts = task.split(' ')
                # Basic example - adjust for more complex operations
                a = float(parts[-3])
                operator = parts[-2]
                b = float(parts[-1])
                for tool in self.tools:
                    if tool.__name__ == "calculator":
                        result = tool(a, b, operator)
                        return result
            except (IndexError, ValueError) as e:
                print(f"Error: Could not parse calculation from task - {e}")
                sys.stdout.flush()

        return "No suitable tool found to respond to the task."


# Tool function
def calculator(a: float, b: float, operator: str) -> str:
    """Perform basic arithmetic operations."""
    print(f"--- Tool 'calculator' called with a={a}, b={b}, operator='{operator}' ---")
    sys.stdout.flush()
    try:
        if operator == '+':
            return str(a + b)
        elif operator == '-':
            return str(a - b)
        elif operator == '*':
            return str(a * b)
        elif operator == '/':
            if b == 0:
                return 'Error: Division by zero'
            return str(a / b)
        else:
            return 'Error: Invalid operator. Please use +, -, *, or /'
    except Exception as e:
        return f'Error: {str(e)}'


# Agent instantiation
print("Creating agent...")
sys.stdout.flush()

calculator_agent: Any = Agent(
    name="calculator_agent",
    model="gemini-pro",  # Hypothetical model
    description="An agent that can perform calculations.",
    instruction="Use the calculator tool to answer math questions.",
    tools=[calculator],  # Add your tool here
)
print("Agent created.")
sys.stdout.flush()
print('\n')

async def main():
    task = "What is the result of 545.34567 * 34555.34"
    response = await calculator_agent.run(task)  # Assuming run is an async method
    print('\n')
    print(f"Task: {task}")
    print(f"Agent Response: {response}")
    sys.stdout.flush()

if __name__ == "__main__":
    await main()

Creating agent...
Agent created.


Agent calculator_agent received task: What is the result of 545.34567 * 34555.34
--- Tool 'calculator' called with a=545.34567, b=34555.34, operator='*' ---


Task: What is the result of 545.34567 * 34555.34
Agent Response: 18844605.0443778
